In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.patches as mpatches
import feyn

# Model agnostic 
from typing import Optional, List, Callable, Dict, Any, List
from pathlib import Path
from utils import ChiefBldr, QLatticeWrapper

This version of Feyn and the QLattice is available for academic, personal, and non-commercial use. By using the community version of this software you agree to the terms and conditions which can be found at https://abzu.ai/eula.

In [2]:
# Get the directory this file lives in
nb_dir = Path.cwd() # notebook directory
project_root = nb_dir.parents[0] # project directory
data_path = project_root / "datasets" / "processed_well_data.csv"

includ_cols = ['Dia', 'Dev(deg)','Area (m2)', 'z','GasDens','LiquidDens', 'P/T','friction_factor', 'critical_film_thickness']
D = ChiefBldr(path=data_path, includ_cols=includ_cols, test_size=0.20)

In [3]:
# define xgboost pipeline
def qlattice(
        hparams: Dict[str,Any]
):
    ql_wrap = QLatticeWrapper(
        feature_tags=includ_cols, 
        **hparams,
    )

    return ql_wrap

hparam_grid = {
            "max_complexity":   [5],
            "n_epochs":         [5],
        }
# train model and optimize hyperparameters via grid search 
trained_model = D.evolv_model(build_model=qlattice, hparam_grid=hparam_grid, k_folds=5)

# output equation 
print(trained_model.express())

Training set R2: 0.5443551700142581
Test set R2: 0.4540572532903502


Training set class score: 0.572289156626506
Test set class score: 0.5476190476190477


-1.44298*(-0.412728*P/T - 1.42342)*tanh(0.82414*Area (m2) + 0.517545) - 0.671326


In [4]:
import pandas as pd


y_pred_scaled = trained_model.predict(D.scaler_X.transform(D.X))
y_pred = D.scaler_y.inverse_transform(y_pred_scaled.reshape(-1, 1)).flatten()

# Assuming y_pred and x1 are 1D arrays or lists of equal length
df = pd.DataFrame({
    'GasFlowrate': D.gsflow,
    'y_pred': y_pred
})

df.to_csv('output_qlattice.csv', index=False)